In [1]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import os
import csv
from pprint import pprint

In [2]:
#listing API urls to use in data pulls
base_url = 'https://public.opendatasoft.com/api/records/1.0/'
records_url_listings = 'search/?dataset=airbnb-listings'
download_url_listings = 'download/?dataset=airbnb-listings'
records_url_reviews = 'search/?dataset=airbnb-reviews'
download_url_reviews = 'download/?dataset=airbnb-reviews'

In [3]:
city_pop_file = 'Resources/city_populations.csv'
city_pop_df = pd.read_csv(city_pop_file)
city_pop_df.head()

,rank,City,Country,population_2019
0,1,Tokyo,Japan,37435192
1,2,Delhi,India,29399140
2,3,Shanghai,China,26317104
3,4,Sao Paulo,Brazil,21846508
4,5,Mexico City,Mexico,21671908


Listing API calls for top 100 cities in world by population

In [169]:
city_list = city_pop_df['City'].head(100)
city_list

0            Tokyo
1            Delhi
2         Shanghai
3        Sao Paulo
4      Mexico City
          ...     
95           Kabul
96           Hefei
97        Tel Aviv
98          Recife
99    Shijiazhuang
Name: City, Length: 100, dtype: object

In [25]:
listing_df = pd.DataFrame()
#city='Austin'
i=0
for city in city_list:
    get_url = base_url+records_url_listings+f'&facet=city&refine.city={city}&rows=10000&fields=country,city,name,host_name,street,property_type,accommodates,bedrooms,bathrooms,beds,price,weekly_price,monthly_price,security_deposit,cleaning_fee,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,listing_url'
    response = requests.get(get_url).json()
    for x in range(len(response['records'])):
        listing_df = listing_df.append(json_normalize(response['records'][x]['fields']))
    print(f'Finished listings in {city}, {i} of {len(city_list)}')
    i+=1
    
    
listing_df.head()

Finished listings in Tokyo, 0 of 100
Finished listings in Delhi, 1 of 100
Finished listings in Shanghai, 2 of 100
Finished listings in Sao Paulo, 3 of 100
Finished listings in Mexico City, 4 of 100
Finished listings in Cairo, 5 of 100
Finished listings in Dhaka, 6 of 100
Finished listings in Mumbai, 7 of 100
Finished listings in Beijing, 8 of 100
Finished listings in Osaka, 9 of 100
Finished listings in Karachi, 10 of 100
Finished listings in Chongqing, 11 of 100
Finished listings in Buenos Aires, 12 of 100
Finished listings in Istanbul, 13 of 100
Finished listings in Kolkata, 14 of 100
Finished listings in Lagos, 15 of 100
Finished listings in Kinshasa, 16 of 100
Finished listings in Manila, 17 of 100
Finished listings in Tianjin, 18 of 100
Finished listings in Rio de Janeiro, 19 of 100
Finished listings in Guangzhou, 20 of 100
Finished listings in Moscow, 21 of 100
Finished listings in Lahore, 22 of 100
Finished listings in Shenzhen, 23 of 100
Finished listings in Bangalore, 24 of 10

,city,bathrooms,name,minimum_nights,country,price,bedrooms,accommodates,beds,number_of_reviews,street,host_name,maximum_nights,listing_url,property_type,review_scores_rating,monthly_price,weekly_price,cleaning_fee,security_deposit
0,Shenzhen,1.0,Bears & Deers Youth Hostel - 3 BunkBed Female ...,1,China,88.0,1.0,6,6.0,0,"福强路, Shenzhen, Guangdong, China",Katniss,1125,https://www.airbnb.com/rooms/13999779,Apartment,NaN,NaN,NaN,NaN,NaN
0,Shenzhen,1.0,welcome gay friends and nudists,1,China,184.0,1.0,1,1.0,1,"沿河南路, Shenzhen, Guangdong, China",Wallace,1125,https://www.airbnb.com/rooms/1631158,Apartment,80.0,NaN,NaN,NaN,NaN
0,Shenzhen,1.0,a Secret sky garden Next to Luohu Perthden Hotel,1,China,264.0,1.0,2,1.0,0,"Shenzhen, Guangdong 518001, China",Oscar,8,https://www.airbnb.com/rooms/14063808,Apartment,NaN,NaN,NaN,NaN,NaN
0,Shenzhen,1.0,宽敞舒适豪华两房,1,China,487.0,2.0,5,2.0,0,"Shenzhen, Guangdong 518031, China",诗凌,1125,https://www.airbnb.com/rooms/14343031,Apartment,NaN,NaN,NaN,NaN,NaN
0,Shenzhen,2.0,福田口岸地铁口，得宝青年公寓女生观景四人间(有独立卫浴),1,China,88.0,1.0,4,4.0,2,"Shenzhen, Guangdong, China",Fanny,1125,https://www.airbnb.com/rooms/13008645,Apartment,90.0,2399.0,624.0,NaN,NaN


In [53]:
listing_df.reset_index(drop=True, inplace = True)

In [58]:
city_id_col=[]
for z in range(len(city_list)):
    for y in range(len(listing_df)):
        if listing_df['city'][y]==city_list[z] :
            city_id_col.append(z)
            
listing_df['city_id']=city_id_col

In [74]:
country_list = listing_df.country.unique()

In [76]:
country_id_col=[]
for w in range(len(country_list)):
    for t in range(len(listing_df)):
        if listing_df['country'][t]==country_list[w] :
            country_id_col.append(w)
            
listing_df['country_id']=country_id_col

In [78]:
listing_df.columns

Index(['city', 'bathrooms', 'name', 'minimum_nights', 'country', 'price',
       'bedrooms', 'accommodates', 'beds', 'number_of_reviews', 'street',
       'host_name', 'maximum_nights', 'listing_url', 'property_type',
       'review_scores_rating', 'monthly_price', 'weekly_price', 'cleaning_fee',
       'security_deposit', 'city_id', 'country_id'],
      dtype='object')

In [79]:
output_df = listing_df.rename(columns = {'name':'listing_name','street':'location_name','accomodates':'capacity','weekly_price':'price_weekly','monthly_price':'price_monthly','security_deposit':'deposit','review_scores_rating':'review_rating'})
output_df.head()

,city,bathrooms,listing_name,minimum_nights,country,price,bedrooms,accommodates,beds,number_of_reviews,...,maximum_nights,listing_url,property_type,review_rating,price_monthly,price_weekly,cleaning_fee,deposit,city_id,country_id
0,Shenzhen,1.0,Bears & Deers Youth Hostel - 3 BunkBed Female ...,1,China,88.0,1.0,6,6.0,0,...,1125,https://www.airbnb.com/rooms/13999779,Apartment,NaN,NaN,NaN,NaN,NaN,23,0
1,Shenzhen,1.0,welcome gay friends and nudists,1,China,184.0,1.0,1,1.0,1,...,1125,https://www.airbnb.com/rooms/1631158,Apartment,80.0,NaN,NaN,NaN,NaN,23,0
2,Shenzhen,1.0,a Secret sky garden Next to Luohu Perthden Hotel,1,China,264.0,1.0,2,1.0,0,...,8,https://www.airbnb.com/rooms/14063808,Apartment,NaN,NaN,NaN,NaN,NaN,23,0
3,Shenzhen,1.0,宽敞舒适豪华两房,1,China,487.0,2.0,5,2.0,0,...,1125,https://www.airbnb.com/rooms/14343031,Apartment,NaN,NaN,NaN,NaN,NaN,23,0
4,Shenzhen,2.0,福田口岸地铁口，得宝青年公寓女生观景四人间(有独立卫浴),1,China,88.0,1.0,4,4.0,2,...,1125,https://www.airbnb.com/rooms/13008645,Apartment,90.0,2399.0,624.0,NaN,NaN,23,0


In [81]:
output_df.drop(['city','country'], inplace = True, axis = 1)

In [93]:
listing_id_col=[]
z=0
for z in range(len(output_df)):
    listing_id_col.append(int(''.join(filter(str.isdigit, output_df.listing_url[z]))))
    
output_df['listing_id'] = listing_id_col

In [107]:
output_df.head(15)

,listing_id,country_id,city_id,listing_name,property_type,location_name,host_name,capacity,bedrooms,beds,price,price_weekly,price_monthly,deposit,cleaning_fee,minimum_nights,maximum_nights,number_of_reviews,review_rating,listing_url
0,13999779,0,23,Bears & Deers Youth Hostel - 3 BunkBed Female ...,Apartment,"福强路, Shenzhen, Guangdong, China",Katniss,NaN,1.0,6.0,88.0,NaN,NaN,NaN,NaN,1,1125,0,NaN,https://www.airbnb.com/rooms/13999779
1,1631158,0,23,welcome gay friends and nudists,Apartment,"沿河南路, Shenzhen, Guangdong, China",Wallace,NaN,1.0,1.0,184.0,NaN,NaN,NaN,NaN,1,1125,1,80.0,https://www.airbnb.com/rooms/1631158
2,14063808,0,23,a Secret sky garden Next to Luohu Perthden Hotel,Apartment,"Shenzhen, Guangdong 518001, China",Oscar,NaN,1.0,1.0,264.0,NaN,NaN,NaN,NaN,1,8,0,NaN,https://www.airbnb.com/rooms/14063808
3,14343031,0,23,宽敞舒适豪华两房,Apartment,"Shenzhen, Guangdong 518031, China",诗凌,NaN,2.0,2.0,487.0,NaN,NaN,NaN,NaN,1,1125,0,NaN,https://www.airbnb.com/rooms/14343031
4,13008645,0,23,福田口岸地铁口，得宝青年公寓女生观景四人间(有独立卫浴),Apartment,"Shenzhen, Guangdong, China",Fanny,NaN,1.0,4.0,88.0,624.0,2399.0,NaN,NaN,1,1125,2,90.0,https://www.airbnb.com/rooms/13008645
5,13625819,0,23,海友酒店深圳罗湖口岸火车站店,House,"Shenzhen, Guangdong 518010, China",凯,NaN,1.0,1.0,200.0,NaN,NaN,NaN,NaN,1,1125,0,NaN,https://www.airbnb.com/rooms/13625819
6,487197,0,23,Deluxe Apt w/ Stunning Seaview,Apartment,"Shenzhen, Guangdong 518000, China",Angelina,NaN,1.0,4.0,NaN,NaN,27959.0,NaN,NaN,1,1125,0,NaN,https://www.airbnb.com/rooms/487197
7,8294832,0,23,Master bed in modern expat Apt,Apartment,"Shenzhen, Guangdong 518000, China",Megan,NaN,1.0,1.0,192.0,NaN,3994.0,NaN,80.0,2,90,1,100.0,https://www.airbnb.com/rooms/8294832
8,11742243,0,23,贴近罗湖口岸的房间，豪华双人房,Apartment,"Shenzhen, Guangdong 518000, China",Xingfan,NaN,1.0,1.0,959.0,NaN,NaN,NaN,NaN,1,1125,0,NaN,https://www.airbnb.com/rooms/11742243
9,12787872,0,23,福田口岸地铁口，得宝青年公寓男生观景四人间,Apartment,"Shenzhen, Guangdong 518026, China",Fanny,NaN,1.0,4.0,80.0,NaN,NaN,NaN,NaN,1,1125,3,100.0,https://www.airbnb.com/rooms/12787872


In [99]:
output_df = output_df.reindex(['listing_id','country_id','city_id','listing_name','property_type','location_name','host_name','capacity','bedrooms','beds','price','price_weekly','price_monthly','deposit','cleaning_fee','minimum_nights','maximum_nights','number_of_reviews','review_rating','listing_url'],axis=1)

In [101]:
listing_output_file = 'Resources/listings.csv'

In [103]:
output_df.to_csv(listing_output_file, index=False, header=True)

Review API calls

In [ ]:
id = listing_id_col[0]
get_url = base_url+records_url_reviews+f'&facet=listing_id&refine.city={id}&rows=10000&fields=listing_id,id,date,comments'
response = requests.get(get_url).json()


In [116]:
review_df = pd.DataFrame()
j=0
for row in listing_id_col:
    get_url = base_url+records_url_reviews+f'&facet=listing_id&refine.listing_id={row}&rows=100&fields=listing_id,id,date,comments'
    response = requests.get(get_url).json()
    for x in range(len(response['records'])):
        review_df = review_df.append(json_normalize(response['records'][x]['fields']))
    if j%50==0:
        print(f'Finished review {j} of {len(listing_id_col)}')
    j+=1
    
review_df.head()

Finished review 0 of 65486
Finished review 50 of 65486
Finished review 100 of 65486
Finished review 150 of 65486
Finished review 200 of 65486
Finished review 250 of 65486
Finished review 300 of 65486
Finished review 350 of 65486
Finished review 400 of 65486
Finished review 450 of 65486
Finished review 500 of 65486
Finished review 550 of 65486
Finished review 600 of 65486
Finished review 650 of 65486
Finished review 700 of 65486
Finished review 750 of 65486
Finished review 800 of 65486
Finished review 850 of 65486
Finished review 900 of 65486
Finished review 950 of 65486
Finished review 1000 of 65486
Finished review 1050 of 65486
Finished review 1100 of 65486
Finished review 1150 of 65486
Finished review 1200 of 65486
Finished review 1250 of 65486
Finished review 1300 of 65486
Finished review 1350 of 65486
Finished review 1400 of 65486
Finished review 1450 of 65486
Finished review 1500 of 65486
Finished review 1550 of 65486
Finished review 1600 of 65486
Finished review 1650 of 65486
Fin

KeyboardInterrupt: 

In [117]:
output_df_review = review_df.reindex(['id','listing_id','date','comments'], axis=1)

In [118]:
review_output_file = 'Resources/reviews.csv'

In [119]:
output_df_review.to_csv(review_output_file, index=False, header=True)

In [167]:
city_unique = listing_df['city'].unique()

In [120]:
city_pop_df.head()

,rank,City,Country,population_2019
0,1,Tokyo,Japan,37435192
1,2,Delhi,India,29399140
2,3,Shanghai,China,26317104
3,4,Sao Paulo,Brazil,21846508
4,5,Mexico City,Mexico,21671908


In [125]:
country_pop_file = 'Resources/country_populations.csv'
country_pop_df = pd.read_csv(country_pop_file, encoding='ISO-8859-1')
country_pop_df.head()

,Rank,Country,Population
0,1,China,"1,421,021,794"
1,2,India,"1,338,676,779"
2,3,United States of America,"325,084,758"
3,4,Indonesia,"264,650,969"
4,5,Pakistan,"207,906,210"


In [127]:
country_gdp_file = 'Resources/country_gdp.csv'
country_gdp_df = pd.read_csv(country_gdp_file, encoding='ISO-8859-1')
country_gdp_df.head()

,Rank,Country,GDP
0,1,Liechtenstein,"$139,100"
1,2,Qatar,"$124,500"
2,3,Monaco,"$115,700"
3,4,Macau,"$111,600"
4,5,Luxembourg,"$106,300"


In [129]:
city_gdp_file = 'Resources/city_gdp.csv'
city_gdp_df = pd.read_csv(city_gdp_file, encoding='ISO-8859-1')
city_gdp_df.head()

,City,GDP per capita
0,San Francisco (Greater),94699
1,Luxembourg,88312
2,Seattle,78262
3,Boston,78183
4,New York (Greater),74244


In [174]:
city_list_df=pd.DataFrame(city_list)
city_list_df

,City
0,Tokyo
1,Delhi
2,Shanghai
3,Sao Paulo
4,Mexico City
...,...
95,Kabul
96,Hefei
97,Tel Aviv
98,Recife


In [183]:
city_list_df.loc[city_list_df['City']=='Sao Paulo'].index[0]

3

In [131]:
country_list

array(['China', 'France', 'United Kingdom', 'Hong Kong', 'Spain',
       'Canada', 'Australia', 'Italy'], dtype=object)

In [168]:
city_df = pd.DataFrame(city_unique)
city_df.rename(columns={'City':'city_name'}, inplace = True)

In [192]:
city_id_list=[]
for x in range(len(city_list)):
    for y in range(len(city_df)):
        if city_df[0][y]==city_list[x]:
            city_id_list.append(x)

In [193]:
city_df['id']= city_id_list

In [282]:
city_pop_list=[]
for city in city_df[0]:
    print(city)
    city_pop_list.append(city_pop_df.loc[city_pop_df['City']==city]['population_2019'].values)

for x in range(len(city_pop_list)):
    city_pop_list[x]=city_pop_list[x][0]    

Shenzhen
Paris
London
Hong Kong
Madrid
Toronto
Barcelona
Qingdao
Alexandria
Melbourne
Sydney
Rome
Montreal


In [331]:
city_df['population']=city_pop_list
city_df

,0,id,population
0,Shenzhen,23,12128721
1,Paris,25,10958187
2,London,34,9176530
3,Hong Kong,45,7490776
4,Madrid,55,6559041
5,Toronto,59,6139404
6,Barcelona,64,5541127
7,Qingdao,66,5499117
8,Alexandria,71,5182450
9,Melbourne,77,4870388


In [161]:
city_pop_df.loc[city_pop_df['City']=='Tokyo']['Country'][0]

'Japan'

In [295]:
city_country_id_list=[]
for city in city_df[0]:
    print(city)
    city_country_id_list.append(city_pop_df.loc[city_pop_df['City']==city]['Country'].values[0])

Shenzhen
Paris
London
Hong Kong
Madrid
Toronto
Barcelona
Qingdao
Alexandria
Melbourne
Sydney
Rome
Montreal


In [328]:
country_id_list=[]
for x in range(len(city_country_id_list)):
    for y in range(len(country_list)):
        if city_country_id_list[x]==country_list[y]:
            country_id_list.append(y)

In [329]:
country_id_list

[0, 1, 2, 3, 4, 5, 4, 0, 6, 6, 7, 5]

In [330]:
city_country_id_list

['China',
 'France',
 'United Kingdom',
 'Hong Kong',
 'Spain',
 'Canada',
 'Spain',
 'China',
 'Egypt',
 'Australia',
 'Australia',
 'Italy',
 'Canada']

In [327]:
country_list

array(['China', 'France', 'United Kingdom', 'Hong Kong', 'Spain',
       'Canada', 'Australia', 'Italy'], dtype=object)